In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from fastai.tabular import *
import seaborn as sns
import math
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['X_train.csv', 'sample_submission.csv', 'X_test.csv', 'y_train.csv']


In [2]:
train_df_X = pd.read_csv('../input/X_train.csv',sep=',')
train_df_Y = pd.read_csv('../input/y_train.csv')
test_df_X = pd.read_csv('../input/X_test.csv')

train_df_X.shape, train_df_Y.shape, test_df_X.shape

((487680, 13), (3810, 3), (488448, 13))

In [4]:
train_df_X.isnull().sum()

row_id                   0
series_id                0
measurement_number       0
orientation_X            0
orientation_Y            0
orientation_Z            0
orientation_W            0
angular_velocity_X       0
angular_velocity_Y       0
angular_velocity_Z       0
linear_acceleration_X    0
linear_acceleration_Y    0
linear_acceleration_Z    0
dtype: int64

In [5]:
train_df_X.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410


In [6]:
train_df_Y.head()

,series_id,group_id,surface
0,0,13,fine_concrete
1,1,31,concrete
2,2,20,concrete
3,3,31,concrete
4,4,22,soft_tiles


In [7]:
test_df_X.head()

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.025773,-0.98864,-0.14801,0.003350,-0.006524,-0.001071,-0.027390,0.10043,4.2061,-5.5439
1,0_1,0,1,-0.025683,-0.98862,-0.14816,0.003439,-0.113960,0.083987,-0.060590,-0.70889,3.9905,-8.0273
2,0_2,0,2,-0.025617,-0.98861,-0.14826,0.003571,-0.080518,0.114860,-0.037177,1.45710,2.2828,-11.2990
3,0_3,0,3,-0.025566,-0.98862,-0.14817,0.003609,0.070067,0.033820,-0.035904,0.71096,1.8582,-12.2270
4,0_4,0,4,-0.025548,-0.98866,-0.14792,0.003477,0.152050,-0.029016,-0.015314,3.39960,2.7881,-10.4100


In [8]:
train_df_Y.surface.value_counts()

concrete                  779
soft_pvc                  732
wood                      607
tiled                     514
fine_concrete             363
hard_tiles_large_space    308
soft_tiles                297
carpet                    189
hard_tiles                 21
Name: surface, dtype: int64

In [9]:
train_df_X.isnull().sum()

row_id                   0
series_id                0
measurement_number       0
orientation_X            0
orientation_Y            0
orientation_Z            0
orientation_W            0
angular_velocity_X       0
angular_velocity_Y       0
angular_velocity_Z       0
linear_acceleration_X    0
linear_acceleration_Y    0
linear_acceleration_Z    0
dtype: int64

In [10]:
train_df_X.shape[0]/128, train_df_Y.shape,test_df_X.shape[0]/128,

(3810.0, (3810, 3), 3816.0)

In [11]:
train_df_X.shape

(487680, 13)

In [12]:
train_df_X.describe()

,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
count,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000
mean,1904.500000,63.500000,-0.018050,0.075062,0.012458,-0.003804,0.000178,0.008338,-0.019184,0.129281,2.886468,-9.364886
std,1099.853353,36.949327,0.685696,0.708226,0.105972,0.104299,0.117764,0.088677,0.229153,1.870600,2.140067,2.845341
min,0.000000,0.000000,-0.989100,-0.989650,-0.162830,-0.156620,-2.371000,-0.927860,-1.268800,-36.067000,-121.490000,-75.386000
25%,952.000000,31.750000,-0.705120,-0.688980,-0.089466,-0.106060,-0.040752,-0.033191,-0.090743,-0.530833,1.957900,-10.193000
50%,1904.500000,63.500000,-0.105960,0.237855,0.031949,-0.018704,0.000084,0.005412,-0.005335,0.124980,2.879600,-9.365300
75%,2857.000000,95.250000,0.651803,0.809550,0.122870,0.097215,0.040527,0.048068,0.064604,0.792263,3.798800,-8.522700
max,3809.000000,127.000000,0.989100,0.988980,0.155710,0.154770,2.282200,1.079100,1.387300,36.797000,73.008000,65.839000


In [13]:
from scipy.stats import kurtosis
from scipy.stats import skew

def _kurtosis(x):
    return kurtosis(x)

def CPT5(x):
    den = len(x)*np.exp(np.std(x))
    return sum(np.exp(x))/den

def skewness(x):
    return skew(x)

def SSC(x):
    x = np.array(x)
    x = np.append(x[-1], x)
    x = np.append(x,x[1])
    xn = x[1:len(x)-1]
    xn_i2 = x[2:len(x)]    # xn+1 
    xn_i1 = x[0:len(x)-2]  # xn-1
    ans = np.heaviside((xn-xn_i1)*(xn-xn_i2),0)
    return sum(ans[1:]) 

def wave_length(x):
    x = np.array(x)
    x = np.append(x[-1], x)
    x = np.append(x,x[1])
    xn = x[1:len(x)-1]
    xn_i2 = x[2:len(x)]    # xn+1 
    return sum(abs(xn_i2-xn))
    
def norm_entropy(x):
    tresh = 3
    return sum(np.power(abs(x),tresh))

def SRAV(x):    
    SRA = sum(np.sqrt(abs(x)))
    return np.power(SRA/len(x),2)

def mean_abs(x):
    return sum(abs(x))/len(x)

def zero_crossing(x):
    x = np.array(x)
    x = np.append(x[-1], x)
    x = np.append(x,x[1])
    xn = x[1:len(x)-1]
    xn_i2 = x[2:len(x)]    # xn+1
    return sum(np.heaviside(-xn*xn_i2,0))

def quaternion_to_euler(x, y, z, w):
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.atan2(t3, t4)

    return X, Y, Z

In [14]:
train_df_X.columns

Index(['row_id', 'series_id', 'measurement_number', 'orientation_X',
       'orientation_Y', 'orientation_Z', 'orientation_W', 'angular_velocity_X',
       'angular_velocity_Y', 'angular_velocity_Z', 'linear_acceleration_X',
       'linear_acceleration_Y', 'linear_acceleration_Z'],
      dtype='object')

In [15]:
def create_Feature(org_df):
    df = org_df.copy()
    df['norm_quat'] = (df['orientation_X']**2 + df['orientation_Y']**2 + df['orientation_Z']**2 + df['orientation_W']**2)
    df['mod_quat'] = (df['norm_quat'])**0.5
    df['norm_X'] = df['orientation_X'] / df['mod_quat']
    df['norm_Y'] = df['orientation_Y'] / df['mod_quat']
    df['norm_Z'] = df['orientation_Z'] / df['mod_quat']
    df['norm_W'] = df['orientation_W'] / df['mod_quat']
    
    df['total_angular_velocity'] = (df['angular_velocity_X'] ** 2 + df['angular_velocity_Y'] ** 2 + df['angular_velocity_Z'] ** 2) ** 0.5
    df['total_linear_acceleration'] = (df['linear_acceleration_X'] ** 2 + df['linear_acceleration_Y'] ** 2 + df['linear_acceleration_Z'] ** 2) ** 0.5
    df['acc_vs_vel'] = df['total_linear_acceleration'] / df['total_angular_velocity']
    
    x, y, z, w = df['orientation_X'].tolist(), df['orientation_Y'].tolist(), df['orientation_Z'].tolist(), df['orientation_W'].tolist()
    nx, ny, nz = [], [], []
    for i in range(len(x)):
        xx, yy, zz = quaternion_to_euler(x[i], y[i], z[i], w[i])
        nx.append(xx)
        ny.append(yy)
        nz.append(zz)
    
    df['euler_x'] = nx
    df['euler_y'] = ny
    df['euler_z'] = nz
    
    df['total_angle'] = (df['euler_x'] ** 2 + df['euler_y'] ** 2 + df['euler_z'] ** 2) ** 5
    df['angle_vs_acc'] = df['total_angle'] / df['total_linear_acceleration']
    df['angle_vs_vel'] = df['total_angle'] / df['total_angular_velocity']
    
    ndf=pd.DataFrame()
    df['angular_velocity'] = df['angular_velocity_X'] + df['angular_velocity_Y'] + df['angular_velocity_Z']
    df['linear_acceleration'] = df['linear_acceleration_X'] + df['linear_acceleration_Y'] + df['linear_acceleration_Z']
    df['velocity_to_acceleration'] = df['angular_velocity'] / df['linear_acceleration']
    for c in df.columns[3:]:
        ndf[c+'_min']=df.groupby(['series_id'])[c].min()
        ndf[c+'_max']=df.groupby(['series_id'])[c].max()        
        ndf[c+'_absMax'] =df.groupby(['series_id'])[c].apply(lambda x : np.max(np.abs(x)))
        ndf[c+'_absMin'] =df.groupby(['series_id'])[c].apply(lambda x : np.min(np.abs(x)))
        ndf[c+'_mean']=df.groupby(['series_id'])[c].mean()
        ndf[c+'_mean']=df.groupby(['series_id'])[c].median()
        ndf[c+'_std']=df.groupby(['series_id'])[c].std()
#         ndf[c+'_log']=df.groupby(['series_id'])[c].apply(lambda x : np.log(x))
#         ndf[c+'_maxtomin']=df[c + '_max'] / df[c + '_min']
        ndf[c + '_q25'] = df.groupby(['series_id'])[c].quantile(0.25)
        ndf[c + '_q50'] = df.groupby(['series_id'])[c].quantile(0.5)
        ndf[c + '_q75'] = df.groupby(['series_id'])[c].quantile(0.75)
#         ndf[c + '_CPT5'] = df.groupby(['series_id'])[c].apply(CPT5) 
        ndf[c + '_SSC'] = df.groupby(['series_id'])[c].apply(SSC) 
        ndf[c + '_skewness'] = df.groupby(['series_id'])[c].apply(skewness)
        ndf[c + '_wave_lenght'] = df.groupby(['series_id'])[c].apply(wave_length)
        ndf[c + '_norm_entropy'] = df.groupby(['series_id'])[c].apply(norm_entropy)
        ndf[c + '_SRAV'] = df.groupby(['series_id'])[c].apply(SRAV)
        ndf[c + '_kurtosis'] = df.groupby(['series_id'])[c].apply(_kurtosis) 
        ndf[c + '_mean_abs'] = df.groupby(['series_id'])[c].apply(mean_abs) 
        ndf[c + '_zero_crossing'] = df.groupby(['series_id'])[c].apply(zero_crossing) 
    return ndf.reset_index()

In [16]:
%time train_fe_X = create_Feature(train_df_X)
%time test_fe_X = create_Feature(test_df_X)
train_fe_X.shape, test_fe_X.shape

CPU times: user 5min 6s, sys: 2 s, total: 5min 8s
Wall time: 5min 8s
CPU times: user 5min 5s, sys: 2.39 s, total: 5min 8s
Wall time: 5min 7s


((3810, 477), (3816, 477))

In [17]:
train_fe_X.to_feather('Train_FE')
test_fe_X.to_feather('Test_FE')
train_df_Y.to_feather('Target')
# train_df_X.isnull().sum()

In [37]:
train_fe_X.shape

(3810, 477)

In [39]:
# x.loc[x['series_id']==4]

In [42]:
# train_df = pd.merge(train_fe_X,train_df_Y,how='inner',on='series_id')
# train_df.shape

In [43]:
# train_df.isnull().sum()

In [41]:
# test_fe_X.shape

(3816, 477)

In [ ]:
train_fe_X.to_feather('Train_FE')
test_fe_X.to_feather('Test_FE')
train_df_Y.to_feather('Target')
# train_df_X.isnull().sum()

In [ ]:
# seed=22
# train_samples = train_df.sample(frac=0.8, random_state=seed)
# valid_samples = train_df.drop(train_samples.index)
# train_samples.loc[:,'isValid'] = 0
# valid_samples.loc[:,'isValid'] = 1

In [ ]:
# train_samples.shape,valid_samples.shape

In [ ]:
# valid_idx = valid_samples.index

In [ ]:
# final_train = pd.concat([train_samples,valid_samples]).reset_index(drop=True)
# final_train.shape

In [ ]:
# Creating upsample for smaller category of data

# undersampled_hard_tiles_train = train_samples.loc[(train_samples.surface == 'hard_tiles'),:]
# undersampled_carpet_train = train_samples.loc[(train_samples.surface == 'carpet'),:]
# dominant_class_train = train_samples.loc[ (train_samples.surface != 'hard_tiles') & (train_samples.surface != 'carpet'),:]


# undersampled_hard_tiles_train.shape, undersampled_carpet_train.shape, dominant_class_train.shape

# upsample_hard_tile = pd.concat([undersampled_hard_tiles_train for i in range(0,18)]).reset_index(drop=True)
# upsample_carpet = pd.concat([undersampled_carpet_train for i in range(0,2)]).reset_index(drop=True)
# upsample_hard_tile.shape , upsample_carpet.shape

# Augmenting data by using random noise

# final_train = pd.concat([upsample_hard_tile,upsample_carpet,dominant_class_train,valid_samples]).reset_index(drop=True)
# final_train.loc[:,'randomnoise'] = [random.random() for i in range(0,final_train.copy().shape[0])]
# final_train.shape

In [ ]:
# final_train.head()

In [ ]:
# final_train.surface.value_counts()

In [ ]:
# test_fe_X =  pd.read_feather('Test_FE')
# test_fe_X.shape

In [ ]:
# test_final_X = test_fe_X.copy()
# test_final_X.loc[:,'randomnoise'] = [random.random() for i in range(0,test_fe_X.copy().shape[0])]
# test_final_X.shape

In [ ]:
# test_final_X = test_fe_X.copy()
# test_final_X.shape

In [ ]:
# final_train.columns
# con_var = final_train.columns
# con_var=con_var.drop(labels=['series_id','group_id','surface','isValid'])
# print(con_var)
# dep_var ='surface'
# procs = [FillMissing, Categorify, Normalize]

In [ ]:
# data = (TabularList.from_df(final_train, cont_names=con_var, procs=procs)
#                             .split_by_idx(valid_idx=valid_idx)
#                             .label_from_df(cols=dep_var)
#                             .add_test(TabularList.from_df(test_final_X, cont_names=con_var, procs=procs))
#                             .databunch())

In [ ]:
# data.show_batch(rows=10,ds_type=DatasetType.Valid)

In [ ]:
# len(data.train_ds),len(data.valid_ds), len(data.test_ds)

In [ ]:
# learn = tabular_learner(data, layers=[400,200], metrics=accuracy)

In [ ]:
# learn.lr_find()
# learn.recorder.plot()

In [ ]:
# learn.fit_one_cycle(18, 3e-03)

In [ ]:
# learn.recorder.plot_losses()

In [ ]:
# learn.save('stage-1')

In [ ]:
# learn.load('stage-1')

In [ ]:
# learn.unfreeze()

In [ ]:
# learn.lr_find()
# learn.recorder.plot()

In [ ]:
# learn.fit_one_cycle(10, 3e-05)

In [ ]:
# learn.recorder.plot_losses()

In [ ]:
# learn.save('stage-2')

In [ ]:
# interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
# interp.plot_confusion_matrix()

In [ ]:
# losses,idxs = interp.top_losses()
# len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
# interp.most_confused()

In [ ]:
# learn.load('stage-1')
# interp = ClassificationInterpretation.from_learner(learn)
# interp.plot_confusion_matrix()

In [ ]:
# interp.most_confused()

In [ ]:
# preds = np.argmax(learn.get_preds(ds_type=DatasetType.Test)[0],axis=1)

In [ ]:
# pred_label=[]
# for row in preds:
#     pred_label.append(data.classes[row])
# # pred_label

In [ ]:
# len(pred_label)

In [ ]:
# submission = pd.DataFrame({"series_id": test_final_X.series_id, "surface": pred_label})
# submission.to_csv("submission_DL.csv", index = False)
# submission.head(10)